<a href="https://colab.research.google.com/github/KevinWang676/Retrieval-based-Voice-Conversion-New/blob/main/Sovits_for_Vast_AI_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 在终端里运行 apt install sudo 和 sudo apt-get install build-essential

In [ ]:
# 查看pytorch版本 建议2.0.1+cu118
import torch
print(torch.__version__)

In [ ]:
! nvidia-smi

In [ ]:
! pip install git+https://github.com/nateraw/so-vits-svc-fork@main
! pip install openai-whisper yt-dlp huggingface_hub demucs

In [ ]:
pip install numba==0.56.4

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from huggingface_hub import login

login()

In [ ]:
CHARACTER = "syz"
DO_EXTRACT_VOCALS = True # 是否分离伴奏
MODEL_REPO_ID = "kevinwang676/ai-syz"

In [ ]:
%%writefile data.csv
ytid,start,end
https://www.bilibili.com/video/BV12o4y1W7kn,21,1352

In [ ]:
import subprocess
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
from demucs.pretrained import get_model, DEFAULT_MODEL
from demucs.apply import apply_model
import torch
import csv
import whisper


def download_youtube_clip(
    video_identifier,
    start_time,
    end_time,
    output_filename,
    num_attempts=5,
    url_base="",
    quiet=False,
    force=True,
):
    output_path = Path(output_filename)
    if output_path.exists():
        if not force:
            return output_path
        else:
            output_path.unlink()

    quiet = "--quiet --no-warnings" if quiet else ""
    command = f"""
        yt-dlp {quiet} -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" "{url_base}{video_identifier}"  # noqa: E501
    """.strip()

    attempts = 0
    while True:
        try:
            _ = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError:
            attempts += 1
            if attempts == num_attempts:
                return None
        else:
            break

    if output_path.exists():
        return output_path.exists(), "success"
    else:
        return None



def split_long_audio(model, filepaths, character_name, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir) / character_name
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{character_name}_{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))


def extract_vocal_demucs(model, filename, out_filename, sr=44100, device=None, shifts=1, split=True, overlap=0.25, jobs=0):
    wav, sr = librosa.load(filename, mono=False, sr=sr)
    wav = torch.tensor(wav)
    ref = wav.mean(0)
    wav = (wav - ref.mean()) / ref.std()
    sources = apply_model(
        model,
        wav[None],
        device=device,
        shifts=shifts,
        split=split,
        overlap=overlap,
        progress=True,
        num_workers=jobs
    )[0]
    sources = sources * ref.std() + ref.mean()

    wav = sources[-1]
    wav = wav / max(1.01 * wav.abs().max(), 1)
    wavfile.write(out_filename, rate=sr, data=wav.numpy().T)
    return out_filename


def create_dataset(
    clips_csv_filepath = "data.csv",
    character = "somebody",
    do_extract_vocals = True, # 是否分离伴奏
    whisper_size = "medium",
    # Where raw yt clips will be downloaded to
    dl_dir = "downloads",
    # Where actual data will be organized
    data_dir = "dataset_raw",
    **kwargs
):
    # dl_path = Path(dl_dir) / character

    dl_path = "./downloads/syz/" # 只需手动创建文件夹；（不需要管）文件夹中的命名规则：syz_0000.wav, syz_0001.wav, syz_0002.wav, ...

    # dl_path.mkdir(exist_ok=True, parents=True)
    if do_extract_vocals:
        demucs_model = get_model(DEFAULT_MODEL)

    with Path(clips_csv_filepath).open() as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            outfile_path = dl_path / f"{character}_{i:04d}.wav"
            download_youtube_clip(row['ytid'], row['start'], row['end'], outfile_path)
            if do_extract_vocals:
                extract_vocal_demucs(demucs_model, outfile_path, outfile_path)

    filenames = sorted([str(x) for x in dl_path.glob("*.wav")])
    whisper_model = whisper.load_model(whisper_size)
    split_long_audio(whisper_model, filenames, character, data_dir)

In [ ]:
"""
Here, we override config to have num_workers=0 because
of a limitation in HF Spaces Docker /dev/shm.
"""

import json
from pathlib import Path
import multiprocessing

def update_config(config_file="configs/44k/config.json"):
    config_path = Path(config_file)
    data = json.loads(config_path.read_text())
    data['train']['batch_size'] = 32
    data['train']['eval_interval'] = 500
    data['train']['num_workers'] = multiprocessing.cpu_count()
    data['train']['persistent_workers'] = True
    data['train']['push_to_hub'] = True
    data['train']['repo_id'] = MODEL_REPO_ID # tuple(data['spk'])[0]
    data['train']['private'] = True
    config_path.write_text(json.dumps(data, indent=2, sort_keys=False))

In [ ]:
create_dataset(character=CHARACTER, do_extract_vocals=DO_EXTRACT_VOCALS)
! svc pre-resample
! svc pre-config
! svc pre-hubert -fm crepe
update_config()

In [ ]:
from __future__ import annotations

import os
import re
import warnings
from logging import getLogger
from multiprocessing import cpu_count
from pathlib import Path
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.accelerators import MPSAccelerator, TPUAccelerator
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.strategies.ddp import DDPStrategy
from lightning.pytorch.tuner import Tuner
from torch.cuda.amp import autocast
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter

import so_vits_svc_fork.f0
import so_vits_svc_fork.modules.commons as commons
import so_vits_svc_fork.utils

from so_vits_svc_fork import utils
from so_vits_svc_fork.dataset import TextAudioCollate, TextAudioDataset
from so_vits_svc_fork.logger import is_notebook
from so_vits_svc_fork.modules.descriminators import MultiPeriodDiscriminator
from so_vits_svc_fork.modules.losses import discriminator_loss, feature_loss, generator_loss, kl_loss
from so_vits_svc_fork.modules.mel_processing import mel_spectrogram_torch
from so_vits_svc_fork.modules.synthesizers import SynthesizerTrn

from so_vits_svc_fork.train import VitsLightning, VCDataModule

LOG = getLogger(__name__)
torch.set_float32_matmul_precision("high")


from pathlib import Path

from huggingface_hub import create_repo, upload_folder, login, list_repo_files, delete_file

# if os.environ.get("HF_TOKEN"):
#     login(os.environ.get("HF_TOKEN"))


class HuggingFacePushCallback(pl.Callback):
    def __init__(self, repo_id, private=False, every=100):
        self.repo_id = repo_id
        self.private = private
        self.every = every

    def on_validation_epoch_end(self, trainer, pl_module):
        self.repo_url = create_repo(
            repo_id=self.repo_id,
            exist_ok=True,
            private=self.private
        )
        self.repo_id = self.repo_url.repo_id
        if pl_module.global_step == 0:
            return
        print(f"\n🤗 Pushing to Hugging Face Hub: {self.repo_url}...")
        model_dir = pl_module.hparams.model_dir
        upload_folder(
            repo_id=self.repo_id,
            folder_path=model_dir,
            path_in_repo=".",
            commit_message="🍻 cheers",
            ignore_patterns=["*.git*", "*README.md*", "*__pycache__*"],
        )
        ckpt_pattern = r'^(D_|G_)\d+\.pth$'
        todelete = []
        repo_ckpts = [x for x in list_repo_files(self.repo_id) if re.match(ckpt_pattern, x) and x not in ["G_0.pth", "D_0.pth"]]
        local_ckpts = [x.name for x in Path(model_dir).glob("*.pth") if re.match(ckpt_pattern, x.name)]
        to_delete = set(repo_ckpts) - set(local_ckpts)

        for fname in to_delete:
            print(f"🗑 Deleting {fname} from repo")
            delete_file(fname, self.repo_id)


def train(
    config_path: Path | str, model_path: Path | str, reset_optimizer: bool = False
):
    config_path = Path(config_path)
    model_path = Path(model_path)

    hparams = utils.get_backup_hparams(config_path, model_path)
    utils.ensure_pretrained_model(model_path, hparams.model.get("type_", "hifi-gan"))

    datamodule = VCDataModule(hparams)
    strategy = (
        (
            "ddp_find_unused_parameters_true"
            if os.name != "nt"
            else DDPStrategy(find_unused_parameters=True, process_group_backend="gloo")
        )
        if torch.cuda.device_count() > 1
        else "auto"
    )
    LOG.info(f"Using strategy: {strategy}")

    callbacks = []
    if hparams.train.push_to_hub:
        callbacks.append(HuggingFacePushCallback(hparams.train.repo_id, hparams.train.private))
    if not is_notebook():
        callbacks.append(pl.callbacks.RichProgressBar())
    if callbacks == []:
        callbacks = None

    trainer = pl.Trainer(
        logger=TensorBoardLogger(
            model_path, "lightning_logs", hparams.train.get("log_version", 0)
        ),
        # profiler="simple",
        val_check_interval=hparams.train.eval_interval,
        max_epochs=hparams.train.epochs,
        check_val_every_n_epoch=None,
        precision="16-mixed"
        if hparams.train.fp16_run
        else "bf16-mixed"
        if hparams.train.get("bf16_run", False)
        else 32,
        strategy=strategy,
        callbacks=callbacks,
        benchmark=True,
        enable_checkpointing=False,
    )
    tuner = Tuner(trainer)
    model = VitsLightning(reset_optimizer=reset_optimizer, **hparams)

    # automatic batch size scaling
    batch_size = hparams.train.batch_size
    batch_split = str(batch_size).split("-")
    batch_size = batch_split[0]
    init_val = 2 if len(batch_split) <= 1 else int(batch_split[1])
    max_trials = 25 if len(batch_split) <= 2 else int(batch_split[2])
    if batch_size == "auto":
        batch_size = "binsearch"
    if batch_size in ["power", "binsearch"]:
        model.tuning = True
        tuner.scale_batch_size(
            model,
            mode=batch_size,
            datamodule=datamodule,
            steps_per_trial=1,
            init_val=init_val,
            max_trials=max_trials,
        )
        model.tuning = False
    else:
        batch_size = int(batch_size)
    # automatic learning rate scaling is not supported for multiple optimizers
    """if hparams.train.learning_rate  == "auto":
    lr_finder = tuner.lr_find(model)
    LOG.info(lr_finder.results)
    fig = lr_finder.plot(suggest=True)
    fig.savefig(model_path / "lr_finder.png")"""

    trainer.fit(model, datamodule=datamodule)

if __name__ == '__main__':
    train('configs/44k/config.json', 'logs/44k')

In [ ]:
# train cluster

In [ ]:
! svc train-cluster

In [ ]:
from huggingface_hub import upload_file

upload_file(path_or_fileobj="/content/logs/44k/kmeans.pt", repo_id=MODEL_REPO_ID, path_in_repo="kmeans.pt")